In [1]:
from moccasin import setup_notebook

setup_notebook()


In [2]:
from moccasin.config import get_active_network
import boa
from boa.contracts.abi.abi_contract import ABIContract

STARTING_ETH_BALANCE = int(1000e18)

active_network = get_active_network()
ffactory = active_network.manifest_named("ffactory")
virtual_token = active_network.manifest_named("virtual_token")
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")
weth = active_network.manifest_named("weth")
agent_factory = active_network.manifest_named("agent_factory")


print(ffactory.router())
print(virtual_token)
print(uniswap_swap_router)
print(agent_factory)


0x8292B43aB73EfAC11FAF357419C38ACF448202C5
<virtual_token interface at 0x0b3e328455c4059EEb9e3f84b5543F74E24e7E1b>
<uniswap_swap_router interface at 0x2626664c2603336E57B271c5C0b26F421741e481>
<agent_factory interface at 0x71B8EFC8BCaD65a5D9386D07f2Dff57ab4EAf533>


In [3]:
boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)
weth.deposit(value=STARTING_ETH_BALANCE)
print(weth.balanceOf(boa.env.eoa))


1000000000000000000000


In [4]:
def swap_exact_input_single(
    swap_router,
    token_in_contract,
    token_out_contract,
    amount_in: int,
    amount_out_min: int,
    pool_fee: int = 3000,  # 0.3% fee tier
) -> int:
    """
    Swaps a fixed amount of token_in for a maximum possible amount of token_out

    Args:
        swap_router: ISwapRouter contract
        token_in_contract: Input token contract
        token_out_contract: Output token contract
        amount_in: Exact amount of input token to swap
        pool_fee: Fee tier (default 0.3% = 3000)

    Returns:
        amount_out: Amount of output token received
    """
    # First approve router to spend token
    token_in_contract.approve(swap_router.address, amount_in)

    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }
    amount_out = swap_router.exactInputSingle(
        (
            token_in_contract.address,
            token_out_contract.address,
            pool_fee,
            boa.env.eoa,
            int(amount_in),
            int(amount_out_min),
            0,
        )
    )
    return amount_out

In [5]:
AMOUNT_IN = int(1e18)
amount_out = swap_exact_input_single(
            swap_router=uniswap_swap_router,
            token_in_contract=weth,
            token_out_contract=virtual_token,
            amount_in=AMOUNT_IN,
            amount_out_min=0,
        )

In [6]:
print(virtual_token.balanceOf(boa.env.eoa))

1536488887315522710389


In [7]:
random_agent_token = agent_factory.allTradingTokens(0)
print(random_agent_token)

0x731814e491571A2e9eE3c5b1F7f3b962eE8f4870


In [8]:
from moccasin.config import Contract

# Create a new contract instance using the ABI from aixbt_agent_token but with your specific address
agent_token = Contract(
    "agent_token",  # name can be anything
    random_agent_token,  # your address
    active_network.manifest_named("agent_token")  # use existing ABI
)

# Now you can interact with it
print(f"Token Name: {agent_token.name()}")
print(f"Token Symbol: {agent_token.symbol()}")
print(f"Decimals: {agent_token.decimals()}")
print(f"Total Supply: {agent_token.totalSupply()}")


ImportError: cannot import name 'Contract' from 'moccasin.config' (/home/jess/Coding/projects/virtual_trading_bot/.venv/lib/python3.12/site-packages/moccasin/config.py)